In [ ]:
# AlphaSymbolic - Unified Hybrid System
# -------------------------------------
# Instructions:
# 1. Runtime -> Change runtime type -> T4 GPU
# 2. Run All

!nvidia-smi

# Install dependencies
!pip install gradio torch torchvision torchaudio scipy matplotlib sympy

# Create Directory Structure
import os
os.makedirs('Code/src', exist_ok=True)
os.makedirs('Code/build', exist_ok=True)
os.makedirs('AlphaSymbolic', exist_ok=True)
directories = ['core', 'data', 'search', 'ui', 'utils']
for d in directories:
    os.makedirs(os.path.join('AlphaSymbolic', d), exist_ok=True)


In [ ]:
%%writefile Code/CMakeLists.txt

cmake_minimum_required(VERSION 3.10)
project(SymbolicRegressionGP)

set(CMAKE_CXX_STANDARD 17)

find_package(CUDA)

if(CUDA_FOUND)
    add_definitions(-DUSE_GPU_ACCELERATION_DEFINED_BY_CMAKE)
    enable_language(CUDA)
    set(CMAKE_CUDA_FLAGS "${CMAKE_CUDA_FLAGS} -O3 -arch=sm_75") # T4 is sm_75
    set(SOURCE_FILES 
        src/main.cpp 
        src/GeneticAlgorithm.cpp 
        src/ExpressionTree.cpp 
        src/GeneticOperators.cpp
        src/Fitness.cpp
        src/FitnessGPU.cu
        src/AdvancedFeatures.cpp
    )
else()
    message(WARNING "CUDA not found. Compiling for CPU only.")
    set(SOURCE_FILES 
        src/main.cpp 
        src/GeneticAlgorithm.cpp 
        src/ExpressionTree.cpp 
        src/GeneticOperators.cpp
        src/Fitness.cpp
        src/FitnessGPU.cu # Still included but ifdef'd out inside
        src/AdvancedFeatures.cpp
    )
endif()

add_executable(SymbolicRegressionGP ${SOURCE_FILES})

if(CUDA_FOUND)
    set_target_properties(SymbolicRegressionGP PROPERTIES CUDA_SEPARABLE_COMPILATION ON)
    target_link_libraries(SymbolicRegressionGP ${CUDA_LIBRARIES})
else()
    target_link_libraries(SymbolicRegressionGP pthread)
endif()


In [ ]:
# Compile C++ Engine
%cd Code
!cmake -B build -S . -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j $(nproc)
%cd ..

In [ ]:
# Run AlphaSymbolic
# The binaries are in ../Code/build/
%cd AlphaSymbolic
!python app.py
